# Hands-on Workshop: Extending and Modifying a Power Grid Model DS Grid

## Introduction & Setup
Power Grid Model DS (PGM-DS) is an open-source Python toolkit for modeling electric power grids. It extends the core power-grid-model engine with a user-friendly interface for data science applications​. In PGM-DS, a power network is represented by a Grid dataclass that manages all network components (nodes, lines, transformers, loads, etc.) and ensures their consistency​.

This library provides a graph-based representation of the network and an API to modify the network structure (e.g. adding or removing equipment), which is very useful for simulation studies​. In this workshop, you will perform a series of short exercises to get hands-on experience with two key features of PGM-DS:
- Extending a grid with custom properties – how to subclass PGM-DS data structures to add your own fields.
- Modifying a grid object – how to build and alter a grid (add nodes, lines, etc.) programmatically.

We assume you are comfortable with Python and Jupyter notebooks. Before we begin, make sure you have PGM-DS installed:

```bash
pip install power-grid-model-ds[visualizer]
```

In [ ]:
!pip install power-grid-model-ds[visualizer] --quiet

In [ ]:
# ⚙️ Setup
import numpy as np
from dataclasses import dataclass

from power_grid_model_ds import Grid
from power_grid_model_ds.arrays import NodeArray, LineArray, SymLoadArray, SourceArray
from power_grid_model_ds.enums import NodeType

from numpy.typing import NDArray

## Exercise 1: Extending the Grid with Custom Properties
Context: By default, the PGM-DS Grid includes standard attributes for each component (like node voltage ratings, line impedances, etc.). However, certain project-specific data (for example, simulation results or custom labels) are not included out of the box​. PGM-DS allows you to extend the grid data model by subclassing its array classes to add new columns. In other words, you can inherit the existing functionality and add your own fields​.

In this exercise, we will extend the grid to include an output voltage u for nodes and an output current i_from for lines, which are not present in the basic grid.

### Step 1: Define Extended Node and Line arrays
We create subclasses of NodeArray and LineArray that include the new properties. We'll call them MyNodeArray and MyLineArray. Each subclass defines a class attribute for the new column and (optionally) a default value for that column via a _defaults dictionary.

**⚙️ Assignment**: Create two Array extensions to hold the x, y, u (extending NodeArray) and i_from (extending LineArray) attributes

**💡 Hint**: https://power-grid-model-ds.readthedocs.io/en/stable/examples/model/grid_extensions_examples.html

In [ ]:
# Build your solution here...

In [ ]:
# %load solutions/introduction_1_1_define_array_extensions

### Step 2: Create an Extended Grid class
Now we'll integrate these new arrays into a custom Grid class. We do this by subclassing the PGM-DS Grid and specifying that our grid should use MyNodeArray and MyLineArray instead of the default NodeArray and LineArray. We'll use Python's dataclass to define the new Grid schema:

**⚙️ Assignment**: Create a new grid class that uses the extended arrays.

**💡 Hint**: https://power-grid-model-ds.readthedocs.io/en/stable/examples/model/grid_extensions_examples.html#adding-the-new-arrays-to-the-grid

**💡 Hint**: Make sure to add the `@dataclass` decorator to your grid.

In [ ]:
# Build your solution here...

In [ ]:
# %load solutions/introduction_1_2_define_my_grid

This ExtendedGrid class inherits all the behavior of Grid but with our extended node and line definitions​. Essentially, we've informed the Grid that whenever it creates or manipulates the node or line arrays, it should use the extended versions that include the extra columns.

### Step 3: Initialize an Extended Grid
With the classes defined, let's create an instance of our extended grid. PGM-DS provides a convenient class method Grid.empty() to initialize an empty grid. We'll call this on our ExtendedGrid:

**⚙️ Assignment**: Instantiate an empty extended grid

**💡 Hint**: https://power-grid-model-ds.readthedocs.io/en/stable/examples/model/grid_examples.html#creating-an-empty-grid

In [ ]:
# Build your solution here...

In [ ]:
# %load solutions/introduction_1_3_grid_empty

Verification: To ensure our extended properties exist, you can access the new attributes:

**⚙️ Assignment**: Print some information about the grid.

**💡 Hint**: Be creative! You can use the grid's attributes and methods to get information about the grid. Using `print()` on an array will format it for better readability.

In [ ]:
# Build your solution here...

In [ ]:
# %load solutions/introduction_1_3_grid_verification

Since we haven't added any nodes or lines yet, these arrays are empty

## Exercise 2: Building and Modifying the Grid Structure

Context: The Grid object allows you to add, remove, and manipulate grid components in-memory. We will now construct a simple network step-by-step using our ext_grid. This will demonstrate how to modify a grid object by adding nodes and branches. In practice, you can start with an empty grid and programmatically add substations, lines, transformers, loads, sources, etc., as needed​. We will create a minimal example with two nodes (a source node and a load node) connected by a line, and verify that the grid is updated accordingly. (For brevity, we'll use a very small grid, but the same methods apply to larger networks.)

Note: PGM-DS typically distinguishes node types (e.g., substation vs. regular node) and requires unique IDs for each element. We will manually specify IDs for clarity. The library’s Grid.append() method will be used to add new component records to the grid​.


### Step 1: Add a substation node
First, let's add a substation node to the grid. We create an MyNodeArray with one entry representing the substation. We need to provide at least an id, a rated voltage (u_rated), and a node type.
We will use the enum NodeType.SUBSTATION_NODE for the type.
In this example, we will assign the substation an ID of 101 and a rated voltage of 10500.0 (which could represent 10.5 kV):

**⚙️ Assignment**: Add a substation to the grid.

**💡 Hint**: https://power-grid-model-ds.readthedocs.io/en/stable/examples/model/grid_examples.html#adding-substations

In [ ]:
# Build your solution here...

In [ ]:
# %load solutions/introduction_2_1_add_substation

Here we constructed a MyNodeArray with one record and then appended it to grid. We set check_max_id=False to disable internal ID checks since we're manually managing IDs in this exercise. After running this, the grid now contains one node. Verification: Check that the node was added. For example:

In [ ]:
print("Nodes in the grid:")
print(grid.node)

### Step 2: Add a second node
Next, we'll add another node to represent a load or another bus in the grid. This node will be of a generic type (we'll use NodeType.UNSPECIFIED, which equals 0 in the enum). We'll give it an ID of 102 and the same base voltage (10.5 kV).

**⚙️ Assignment**: Add a node to the grid. 

In [ ]:
# Build your solution here...

In [ ]:
# %load solutions/introduction_2_2_add_node

In [ ]:
print("Nodes in the grid:")
print(grid.node)

### Step 3: Add a line connecting the two nodes
Now that we have two nodes, we will connect them with a line. We'll use our MyLineArray to create a single line record. We need to specify an ID for the line (let's use 201), the from_node and to_node it connects (101 to 102), and statuses to indicate the line is active. We should also provide line electrical parameters (resistance, reactance, etc.) – we'll use some placeholder values here for demonstration:

**⚙️ Assignment**: Add a line to the grid.

**💡 Hint**: https://power-grid-model-ds.readthedocs.io/en/stable/examples/model/grid_examples.html#adding-lines

In [ ]:
# Build your solution here...

In [ ]:
# %load solutions/introduction_2_3_add_line

This adds a line (ID 201) connecting node 101 to 102. We marked the line as active by setting both from_status and to_status to 1. We also provided some dummy impedance values. The approach of constructing a LineArray (or in our case, MyLineArray) with the necessary fields and appending it to the grid is shown in the official examples​. Verification: Check that the line was added to the grid:

In [ ]:
print(grid.line)

### Step 4: Add a load to the second node
We'll now add a load connected to node 102. PGM-DS uses a SymLoadArray for symmetrical loads. We will create a single load with an ID of 401 at node 102. We need to specify the node it is attached to, a load type code (we'll use 1 for a basic load type), the specified active (p_specified) and reactive (q_specified) power (let's say 1e6 each, representing 1 MW and 1 Mvar for example), and set its status to active (1):

**⚙️ Assignment**: Add a load to the grid.

**💡 Hint**: https://power-grid-model-ds.readthedocs.io/en/stable/examples/model/grid_examples.html#adding-loads

In [ ]:
# Build your solution here...

In [ ]:
# %load solutions/introduction_2_4_add_load

This adds one load to node 102. In practice, adding loads ensures that node 102 will be consuming power in any simulation. Verification: Check that the load appears in the grid:

In [ ]:
print(grid.sym_load)

### Step 5: Add a source to the substation node
Finally, we'll add a power source to supply the grid at the substation (node 101). We'll use SourceArray for this. We'll create a source with ID 501 at node 101, status active (1), and set a reference voltage u_ref. Typically, u_ref might be the slack/reference voltage magnitude or angle; we'll use 0.0 as a reference angle (assuming the default usage):

**⚙️ Assignment**: Add a source to the grid.

**💡 Hint**: https://power-grid-model-ds.readthedocs.io/en/stable/examples/model/grid_examples.html#adding-a-source

In [ ]:
# Build your solution here...

In [ ]:
# %load solutions/introduction_2_5_add_source

This adds a source (e.g., a generator or slack source) at node 101, so the grid now has a supply. Verification: Check the source:

In [ ]:
print(grid.source)

You should see [501] as the source ID and [101] as the node, indicating the source is at node 101. The count of sources should be 1. Now we have built a simple grid with 2 nodes (101 and 102), 1 line connecting them, 1 load at node 102, and 1 source at node 101. It's good practice to ensure all IDs are unique and there are no inconsistencies. PGM-DS provides a method grid.check_ids() to validate this.

**⚙️ Assignment**: Check whether all IDs are correct.

**💡 Hint**: The grid has a method for that!

In [ ]:
# Build your solution here...

In [ ]:
# %load solutions/introduction_2_6_check_ids

If everything is correct, this should execute without errors (it will raise an exception if any duplicate or conflicting IDs were found). We expect no issues since we chose unique IDs for each element type. For a final summary, let's print out the contents of our grid's key components:

In [ ]:
print("Nodes:", grid.node.id)         # Expect [101 102]
print("Lines:", grid.line.id)         # Expect [201]
print("Loads:", grid.sym_load.id)     # Expect [401]
print("Sources:", grid.source.id)     # Expect [501]

Now we can visualize the resulting network

In [ ]:
from power_grid_model_ds.visualizer import visualize

visualize(grid)